https://cookbook.openai.com/examples/assistants_api_overview_python

In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip show openai | grep Version

Version: 1.3.7


In [ ]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
import openai  # for OpenAI API calls
import time  # for measuring time duration of API calls

#from openai import OpenAI
from google.colab import userdata

userdata.get('OPENAI_API_KEY')
openai.api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'created_at': 1701799873,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [ ]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL',
 'created_at': 1701799912,
 'metadata': {},
 'object': 'thread'}

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_xrOaRcTWL0rRqwDc8bEtaKkG',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1701799929,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL'}

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_nOrDOiMm3Zo3scmcrQNakeiz',
 'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701799949,
 'expires_at': 1701800549,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL',
 'tools': []}

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_nOrDOiMm3Zo3scmcrQNakeiz',
 'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'cancelled_at': None,
 'completed_at': 1701799954,
 'created_at': 1701799949,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1701799949,
 'status': 'completed',
 'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL',
 'tools': []}

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_oSZzu2UDR90lSLNmNxYYQZX7',
   'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, to solve the equation `3x + 11 = 14`, first subtract 11 from both sides to get `3x = 3`, then divide both sides by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1701799951,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_nOrDOiMm3Zo3scmcrQNakeiz',
   'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL'},
  {'id': 'msg_xrOaRcTWL0rRqwDc8bEtaKkG',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1701799929,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL'}],
 'object': 'list',
 'first_id': 'msg_oSZzu2UDR90lSL

In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_jIrJSoQCLo3tdhb1abG7mZB2',
   'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! To solve for x, you need to isolate it on one side of the equation. First, subtract 11 from both sides to get rid of the constant term on the side with x:\n\n3x + 11 - 11 = 14 - 11\nThis simplifies to:\n3x = 3\n\nNow divide both sides by 3 to solve for x:\n3x / 3 = 3 / 3\nWhich simplifies to:\nx = 1\n\nSo the solution is x = 1.'},
     'type': 'text'}],
   'created_at': 1701800053,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_xBXTVKr2AgYe0UGHuretBtk7',
   'thread_id': 'thread_EWSRFOc9N42lQ7wZvnGu69TL'}],
 'object': 'list',
 'first_id': 'msg_jIrJSoQCLo3tdhb1abG7mZB2',
 'last_id': 'msg_jIrJSoQCLo3tdhb1abG7mZB2',
 'has_more': False}

In [ ]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [ ]:
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'created_at': 1701799873,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
        'required': ['question_text']}}},
     'required': ['title', 'questions']},
    'de

In [ ]:
from openai import OpenAI
import openai  # for OpenAI API calls
import time  # for measuring time duration of API calls

#from openai import OpenAI
from google.colab import userdata

userdata.get('OPENAI_API_KEY')
openai.api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)
MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."


def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [ ]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [ ]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [ ]:
show_json(run)

{'id': 'run_TyR6jsSOpdC5SZ427rootZ1D',
 'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701802134,
 'expires_at': 1701802734,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_U56ynaVrGV7KhFGSgHsMK6yc',
     'function': {'arguments': '{\n  "title": "Math Concepts Quiz",\n  "questions": [\n    {\n      "question_text": "Explain in your own words what a prime number is.",\n      "question_type": "FREE_RESPONSE"\n    },\n    {\n      "question_text": "Which of the following numbers is a prime number?",\n      "question_type": "MULTIPLE_CHOICE",\n      "choices": ["15", "48", "79", "100"]\n    }\n  ]\n}',
      'name': 'display_quiz'},
     'type': 'function'}]},
  'type': 'sub

In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Concepts Quiz',
 'questions': [{'question_text': 'Explain in your own words what a prime number is.',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'Which of the following numbers is a prime number?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['15', '48', '79', '100']}]}

In [ ]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Concepts Quiz

Explain in your own words what a prime number is.

Which of the following numbers is a prime number?
0. 15
1. 48
2. 79
3. 100

Responses: ["I don't know.", 'a']


In [ ]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [ ]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'created_at': 1701799873,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
        'required': ['question_text']}}},
     'required': ['title', 'questions']},
    'de

In [ ]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [ ]:
show_json(run)

{'id': 'run_mtKWrskHIA8QsdsjaLFBuDo0',
 'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701802395,
 'expires_at': 1701802995,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_1d4fJXMwjPMcLsa3w19LUhDW',
     'function': {'arguments': '{\n  "title": "Math Concepts Quiz",\n  "questions": [\n    {\n      "question_text": "Describe what a prime number is.",\n      "question_type": "FREE_RESPONSE"\n    },\n    {\n      "question_text": "What is the smallest prime number?",\n      "question_type": "MULTIPLE_CHOICE",\n      "choices": ["0", "1", "2", "3"]\n    }\n  ]\n}',
      'name': 'display_quiz'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 17

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Concepts Quiz

Explain in your own words what a prime number is.

Which of the following numbers is a prime number?
0. 15
1. 48
2. 79
3. 100

Responses: ["I don't know.", 'a']


In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Concepts Quiz',
 'questions': [{'question_text': 'Describe what a prime number is.',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'What is the smallest prime number?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['0', '1', '2', '3']}]}

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Concepts Quiz

Describe what a prime number is.

What is the smallest prime number?
0. 0
1. 1
2. 2
3. 3

Responses: ["I don't know.", 'a']


In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_mtKWrskHIA8QsdsjaLFBuDo0',
 'assistant_id': 'asst_atGhgUu6Ui8pvHKfio2gxEJo',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701802395,
 'expires_at': 1701802995,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1701802396,
 'status': 'queued',
 'thread_id': 'thread_rm4eakTaszT18WEyyANsIowV',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_t

In [ ]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: The solution to the equation \(3x + 11 = 14\) is \(x = 1\).

# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is a branch of mathematics focusing on the study of vectors, vector spaces (also known as linear spaces), linear transformations (functions that preserve vector addition and scalar multiplication), and systems of linear equations. Fundamental concepts in linear algebra include:

- **Vectors:** Quantities with both magnitude and direction, represented by coordinates that define their position in space relative to an origin.
- **Vector Spaces:** Collections of vectors that can be scaled and added together, following certain mathematical rules (axioms).
- **Matrices:** Rectangular arrays of numbers (or functions) that can represent linear transformations, systems of linear equations, or coefficients that relate vectors.
- **Linear Transformations:** Functions

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: Thank you for your responses. Here is the feedback:

1. A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself. For instance, 2, 3, 5, and 7 are prime numbers because they can only be divided evenly by 1 and by themselves.

2. The smallest prime number is actually "2", which is the only even prime number because it's divisible by 1 and 2 only. The option "0" and "1" are not prime numbers because prime numbers must be greater than 1, and no number can divide 0 evenly except itself.

